In [1]:
# load t-lite and show it is uncensored
# load aligning russian dataset
# run DPO, plot and cherry-picks
# test t-lite is censored now

In [2]:
from transformers import AutoModelForCasualLM, AutoTokenizer
repo_id='AnatoliiPotapov/T-lite-0.1'
model=AutoModelForCasualLM.from_pretrained(repo_id)
tokenizer=AutoTokenizer.from_pretrained(repo_id)

ModuleNotFoundError: No module named 'transformers'